In [ ]:
from __future__ import print_function, division
from collections import defaultdict, OrderedDict
import gzip
import lz4.frame as lz4f
import cloudpickle as cpkl
import json
import re
import os

import uproot
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from fnal_column_analysis_tools import hist
from fnal_column_analysis_tools.hist import plot, export

In [ ]:
import sys
print(sys.version)

In [ ]:
with lz4f.open("hists.cpkl.lz4") as fin:
    hists_unmapped = cpkl.load(fin)

In [ ]:
process = hist.Cat("process", "Process", sorting='placement')
process_cats = ("dataset")

process_map = OrderedDict()
process_map["QCD"] = [
    "QCD_HT500to700_TuneCP5_13TeV-madgraphMLM-pythia8",
    "QCD_HT700to1000_TuneCP5_13TeV-madgraphMLM-pythia8",
    "QCD_HT1500to2000_TuneCP5_13TeV-madgraphMLM-pythia8",
    "QCD_HT1000to1500_TuneCP5_13TeV-madgraphMLM-pythia8",
    "QCD_HT2000toInf_TuneCP5_13TeV-madgraphMLM-pythia8",
]

hists = {}

for key, val in hists_unmapped.items():
    if isinstance(val, hist.Hist):
        hists[key] = val.group(process, process_cats, process_map)

In [ ]:
histo = hists['ddtmaping_preselection'].sum('AK8Puppijet0_isHadronicV', overflow='all')
val_QCD = histo.values(overflow='allnan')[('QCD',)]
qcd_maxval_temp = np.cumsum(val_QCD, axis=2)
qcd_maxval = qcd_maxval_temp[:,:,-1]

norma = qcd_maxval_temp / np.maximum(1e-10,qcd_maxval[:,:,np.newaxis])
print(norma)

In [ ]:
hist_y_QCD = histo.sum("process")
print(hist_y_QCD)
template = hist_y_QCD.sum("AK8Puppijet0_N2sdb1")
print(template)
hist_y_QCD.clear()
hist_y_QCD._sumw = {():norma}

In [ ]:
import scipy.ndimage as sc
res = np.apply_along_axis(lambda norma: norma.searchsorted(0.26), axis = 2, arr = norma)
print(res.shape)
res[res>100]=0
print(hist_y_QCD.identifiers("AK8Puppijet0_N2sdb1"))
def bineval(a):
    return hist_y_QCD.identifiers("AK8Puppijet0_N2sdb1")[a].lo
print(bineval(55))
binfunc = np.vectorize(bineval)
qmap = binfunc(res)
print(qmap.shape)
smooth_qmap = sc.filters.gaussian_filter(qmap,1)

In [ ]:
template.clear()
template._sumw = {():smooth_qmap}
template.label = 'N2 cut at 26%'

values_nonan = template.values()[()]
print(template)

print(values_nonan)

In [ ]:
fig3, ax3, _ = plot.plot2d(template, xaxis="ak8jet_rho", patch_opts={})

fig3.savefig("plots/N2DDT_26_2018bits_NoSmoothing.pdf")
fig3.savefig("plots/N2DDT_26_2018bits_NoSmoothing.png")
fig3.savefig("plots/N2DDT_26_2018bits_NoSmoothing.svg")

In [ ]:
import ROOT
outfile = ROOT.TFile("plots/n2ddtmap_2018bits_Gaussian1Sigma.root","recreate")
outfile.cd()
print(values_nonan.shape)
h1 = ROOT.TH2F("h1","h1",52, -6, -2.1, 100, 300, 1300)
for i in range(h1.GetNbinsX()):
    for j in range(h1.GetNbinsY()):
        h1.SetBinContent(i+1,j+1,values_nonan[j][i])
h1.Write()
outfile.Close()